In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

In [3]:
indices = pd.MultiIndex.from_frame(pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'locations.csv'), usecols=['Time', 'Vehicle_Number_Plate'], parse_dates=['Time'])).reorder_levels([1,0])
area_centers = pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'areas.csv'), index_col=0)
demand = pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'demand.csv'), index_col=0)
locations = pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'locations.csv'), parse_dates=['Time'])
actions = pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'actions.csv'), parse_dates=['Time'])

In [4]:
demand.index = pd.to_datetime(demand.index, format='%Y-%m-%d %H:%M')
locations.index = pd.MultiIndex.from_frame(locations.loc[:,['Time', 'Vehicle_Number_Plate']])
actionsi = pd.MultiIndex.from_frame(actions.loc[:,['Time', 'Vehicle_Number_Plate']])
actions.index = actionsi
locations.drop(labels=['Time', 'Vehicle_Number_Plate'], axis=1, inplace=True)
actions.drop(labels=['Time', 'Vehicle_Number_Plate'], axis=1, inplace=True)
vehicle_counts = locations.loc[:, locations.columns[locations.columns.str.contains('Zone')]].groupby('Time').sum()

In [25]:
print('Indices')
it = indices.to_list()
%timeit it[1000]
%timeit indices[1000]
print('Locations')
vloc=locations.values
lloc=list(vloc)
tloc=tuple(vloc)
%timeit torch.tensor(locations.loc[indices[1000]].values)
%timeit torch.tensor(locations.loc[it[1000][0], it[1000][1]].values)
%timeit torch.tensor(locations.iloc[1000].values)
%timeit torch.tensor(vloc[1000])
%timeit torch.tensor(lloc[1000])
%timeit torch.tensor(tloc[1000])
print('Actions')
at = actionsi.to_list()
%timeit indices[1000] in actionsi
%timeit indices[1000] in at
%timeit it[1000] in at
print('Demand')
dt = demand.to_dict('index')
%timeit torch.tensor(demand.loc[indices[1000][0]].values)
%timeit torch.tensor(tuple(dt[indices[1000][0]].values()))
%timeit torch.tensor(tuple(dt[it[1000][0]].values()))
print('Counts')
vc = vehicle_counts.to_dict('index')
%timeit torch.tensor(vehicle_counts.loc[indices[1000][0]].values)
%timeit torch.tensor(tuple(vc[indices[1000][0]].values()))
%timeit torch.tensor(tuple(vc[it[1000][0]].values()))

Indices
46.3 ns ± 4.77 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
25.2 µs ± 6.26 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Locations
320 µs ± 17.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
365 µs ± 104 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
251 µs ± 84.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
8.6 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
7.66 µs ± 654 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
7.2 µs ± 183 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Actions
57 µs ± 2.45 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
699 µs ± 25.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
637 µs ± 12.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Demand
205 µs ± 16.7 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
41.8 µs ± 2.09 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
8.33 µs ± 1.16 µs